In [19]:
#Importing the libraries to be used:
import requests
import io
import sklearn
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, preprocessing
import numpy.polynomial.polynomial as poly
%matplotlib inline

In [22]:
url = 'https://www.kaggle.com/mlg-ulb/creditcardfraud/download'
r = requests.get(url).text
df = pd.read_csv(io.StringIO(r))
#df = pd.read_csv(r)
#print(df.shape)

TypeError: initial_value must be str or None, not bytes

In [8]:
df.dropna('columns')
print(df.shape)

(2, 1)
